In [1]:
import h5py
import numpy as np
import pandas as pd

In [4]:
root_path = "original_data/"

# y_train = pd.read_csv(root_path + "y_train.csv").as_matrix()[:, 1].squeeze()
# X_train = h5py.File(root_path + "X_train.h5", "r")

y_test = pd.read_csv(root_path + "y_test.csv").values[:, 1].squeeze()
X_test = h5py.File(root_path + "X_test.h5", "r")

In [9]:
df_test = pd.DataFrame(data=X_test["features"][:])

In [10]:
df_test.shape

(238366, 1261)

In [11]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260
0,213.0,145.954058,227.901408,137.092758,184.0,3.0,2742.0,2940.0,2670.0,120.0,...,16.796756,9.981370,1.507478,-5.245314,-11.189450,-17.122052,-21.020151,-20.757347,-17.367410,-12.884581
1,166.0,144.122566,225.313253,131.541901,370.0,3.0,2417.0,2370.0,1980.0,120.0,...,29.113856,20.194461,12.018300,4.291115,-4.943849,-16.715191,-25.488310,-30.634161,-36.146962,-41.065591
2,423.0,149.903158,249.645390,311.165811,326.0,2.0,12072.0,4080.0,7110.0,120.0,...,-23.920095,-10.636700,11.657997,25.718743,33.229645,34.339333,29.847606,24.619604,15.236349,11.382313
3,113.0,142.651369,227.654867,200.032274,257.0,3.0,2082.0,1950.0,930.0,0.0,...,36.702652,30.363519,24.929158,19.713275,12.165508,2.269518,-8.760884,-19.172103,-28.412247,-37.208437
4,158.0,143.938660,225.088608,157.963681,179.0,3.0,2374.0,2370.0,1740.0,120.0,...,7.168394,3.291635,-1.047935,-3.962133,-3.315331,-3.567024,-3.789086,-2.363467,-5.025818,-11.593094


In [12]:
def label_columns(df):
    l_columns = [
        'num_pso', 
        'mean_amp_pso',
        'mean_dur_pso',
        'amp_cso',
        'dur_cso',
        'curr_sleep_stage',
        'time_since_sleep',
        'time_in_ds',
        'time_in_ls',
        'time_in_rs',
        'time_in_ws'
    ]
    
    for i in range(12, 1261+1):
        l_columns.append('eeg_signal_%s'%(i-12+1))
    
    df.columns = l_columns

In [13]:
# Label columns
label_columns(df_test)

# Add dummies
df_test = pd.get_dummies(
    df_test,
    prefix=['curr_sleep_stage'],
    columns=['curr_sleep_stage']
)

# Add outcome variable 'SO'
y_test_df = pd.DataFrame(
    y_test,
    columns=['SO'],
    index=df_test.index  #!important to keep same index
)
df_test['SO'] = y_test_df['SO']


df_test.head()

,num_pso,mean_amp_pso,mean_dur_pso,amp_cso,dur_cso,time_since_sleep,time_in_ds,time_in_ls,time_in_rs,time_in_ws,...,eeg_signal_1244,eeg_signal_1245,eeg_signal_1246,eeg_signal_1247,eeg_signal_1248,eeg_signal_1249,eeg_signal_1250,curr_sleep_stage_2.0,curr_sleep_stage_3.0,SO
0,213.0,145.954058,227.901408,137.092758,184.0,2742.0,2940.0,2670.0,120.0,630.0,...,-5.245314,-11.189450,-17.122052,-21.020151,-20.757347,-17.367410,-12.884581,0,1,0
1,166.0,144.122566,225.313253,131.541901,370.0,2417.0,2370.0,1980.0,120.0,480.0,...,4.291115,-4.943849,-16.715191,-25.488310,-30.634161,-36.146962,-41.065591,0,1,1
2,423.0,149.903158,249.645390,311.165811,326.0,12072.0,4080.0,7110.0,120.0,1350.0,...,25.718743,33.229645,34.339333,29.847606,24.619604,15.236349,11.382313,1,0,2
3,113.0,142.651369,227.654867,200.032274,257.0,2082.0,1950.0,930.0,0.0,480.0,...,19.713275,12.165508,2.269518,-8.760884,-19.172103,-28.412247,-37.208437,0,1,0
4,158.0,143.938660,225.088608,157.963681,179.0,2374.0,2370.0,1740.0,120.0,480.0,...,-3.962133,-3.315331,-3.567024,-3.789086,-2.363467,-5.025818,-11.593094,0,1,0


In [14]:
df_test.to_hdf('df_test_CNN.h5', key='df_test_CNN')